# Pipeline3.5.1

In [ ]:
from Bio import SeqIO
import numpy as np
from sklearn.metrics import pairwise_distances
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import pandas as pd
pd.options.mode.chained_assignment = None
import igraph as ig
import matplotlib.pyplot as plt
from collections.abc import Iterable
import ete3
from collections import (defaultdict,OrderedDict,Counter)
import scipy.stats as st

In [12]:
        input_file = '/data/judong/Python_rRNA_alingmentMergePipeline/InputUpdatedBash/BashInput/RNA_Genes/HG001_18S_clusters/Combined/HG001_18S_cluster23.fa'
        records = SeqIO.to_dict(SeqIO.parse(input_file, "fasta"))
        ids = list(SeqIO.to_dict(SeqIO.parse(input_file, "fasta")).keys())
        len(ids)

53

In [2]:
def CreateMatrix():
        AcChrList = ['chr13','chr14','chr15','chr21','chr22']
        input_file = '/data/judong/Python_rRNA_alingmentMergePipeline/InputUpdatedBash/BashInput/RNA_Genes/HG001_18S_clusters/Combined/HG001_18S_cluster23.fa'
        records = SeqIO.to_dict(SeqIO.parse(input_file, "fasta"))
        ids = list(SeqIO.to_dict(SeqIO.parse(input_file, "fasta")).keys())
        ids = list(
                    ids[i]
            for i in range(0,len(ids))
                if ids[i][0:5] in AcChrList
        )
        array =[
            y
            for i in range(0,len(ids))
                if ids[i][0:5] in AcChrList
                    for y in records[ids[i]].seq
                    
]
        m = np.array(array)
        M = np.reshape(m, (-1, len(records[ids[1]].seq)))
        M_rows, M_cols = M.shape 
        return M,M_rows,M_cols,ids
def CreateReadingNameArray(ids):
    ReadingNameArray = np.array([ids[i] for i in range(0,len(ids))])
    return ReadingNameArray
def RemoveTerminalGaps(M,M_rows, M_cols):

    ##find all gaps
    gapRowCoords,gapColCoords = np.where(M == '-')
    gapCoords=np.vstack((gapRowCoords,gapColCoords)).T


    #the rows with 1st 3 chars are gaps
    MissInfoRowCoordsF=[
        (i)
    for i,j in gapCoords
        if j==0 and M[i,(j+1)]=='-' and M[i,(j+2)]=='-'
]    

    #the rows with last 3 chars are gaps
    MissInfoRowCoordsB=[
        (i)
        for i,j in gapCoords
            if j==M_cols-1 and M[i,(j-2)]=='-'and M[i,(j-3)]=='-'
] 

    #conbine Fw and Bw
    MissInfoRowCoords=list(set(MissInfoRowCoordsF + MissInfoRowCoordsB))
        # mask rows with terminal gaps 
    for i in MissInfoRowCoordsF:
        for j in range(0,M_cols):
            if M[i,j]== '-':
                M[i,j] = '+'
            if M[i,j+1]!= '-':
                break
    for i in MissInfoRowCoordsB:
        for j in reversed(range(0,M_cols)):
            if M[i,j]== '-':
                M[i,j] = '+'
            if M[i,j-1]!= '-':
                break
   
    M_rows,M_cols = M.shape
    print(len(MissInfoRowCoords)," rows reads with terminal gaps have been Masked.")
    return M,M_rows,M_cols,MissInfoRowCoords
def CreateReadingNameDicts(ReorderRN):
    ReadingDicts = {}
    for i in range(0,len(ids)):
        ReadingDicts[i] = [ReorderRN[i]]
    return ReadingDicts
def CreateReadingNameDicts(ReorderRN):
    ReadingDicts = {}
    for i in range(0,len(ids)):
        ReadingDicts[i] = [ReorderRN[i]]
    
    return ReadingDicts
def ErrorMasking(M,M_rows,M_cols,T):

    for x in range(0,M_cols):
        ele_array = np.unique(M[:,x])
    
        if len(ele_array) == 1:
                pass
        else:
               for i in ele_array:
                ele_per = ((M[:,x]==i).sum(axis=0))/M_rows
           
                if ele_per > T:
                    pass
                elif ele_per <= T and ((M[:,x]=='-').sum(axis=0))>(M_rows*(9/10)):
                    for j in (np.where(M[:,x]==i)):
                        M[j,x]='+'
                else:
                    for j in (np.where(M[:,x]==i)):
                        M[j,x]='n'
def StringMatrixToNumber (M1):
        
        M2=M1

        M2[M2 == 'A']=int("1")
        M2[M2 == 'C']=int("2")
        M2[M2 == 'T']=int("3")
        M2[M2 == 'G']=int("4")
        M2[M2 == 'n']=int("0")
        M2[M2 == '-']=int("5")#-2
        M2[M2 == '+']=int("6")#-1
        Int = np.vectorize(lambda x: int(x))
        M2 = Int(M1)   
        M2[M2 == 5]=-2
        M2[M2 == 6]=-1
        print("First three position of number Matrix:",M2[0,0],M2[0,1],M2[0,2],"Datatype:",M2.dtype)
        return M2
def CalculatePairWiseDistance_sk(u,v):
    a = np.bitwise_and(np.bitwise_and(u > 0,v > 0),(u!=v)).sum(axis=0)
    b = np.bitwise_and(u == -2,v>=0).sum(axis=0) + np.bitwise_and(v == -2,u>=0).sum(axis=0)
    return a+b
def CalculatePairWiseDistance(M2,ReadingD):
        M1=M2
        # distance matrix with M_rows* M_rows shape
        Mdis0 = np.full((M_rows,M_rows),-1)
        repeat = np.full(M_rows,1,dtype=bool)
        counter = np.full(M_rows,1,dtype=int)
        nmcounter = 0
        #1.1 If we want to see How Merge happens unhash the code below
        #MergeDicts_Pd = pd.DataFrame( {'SeqID_i':[],  'SeqID_j':[]})
        for i in range(0,M_rows-1):
            if repeat[i]==False:
                    continue
            else:
                for j in range(i+1,M_rows):
                    if repeat[j]==False:
                         continue
                    else:
                        a = np.bitwise_and(np.bitwise_and(M1[i,]>0,M1[j,]>0),(M1[i,]!=M1[j,])).sum(axis=0)
                        b = np.bitwise_and(M1[i,]== -2,M1[j,]>=0).sum(axis=0) + np.bitwise_and(M1[j,]== -2,M1[i,]>=0).sum(axis=0)
                        Mdis0[i,j]= a+b  
                        if a+b == 0:
                            counter[i]+=1
                            counter[j]-=1
                            repeat[j]= False
                            for ele in ReadingD[j]:
                                if ele not in ReadingD[i]:
                                    ReadingD[i].append(ele)
                            #1.1 If we want to see How Merge happens unhash the code below
                            #df2 = pd.DataFrame([[i,j]], columns=['SeqID_i','SeqID_j'])
                            #MergeDicts_Pd = pd.concat([df2, MergeDicts_Pd])
                      #  else:
        return counter,Mdis0,ReadingD#MergeDicts_Pd
def n_correction(Mdis0,M1):    
    M_Rows,M_Cols = M1.shape
    def unique_rows(a):
        a = np.ascontiguousarray(a)
        unique_a = np.unique(a.view([('', a.dtype)]*a.shape[1]))
        return unique_a.view(a.dtype).reshape((unique_a.shape[0], a.shape[1]))
    
    r,c = np.where(Mdis0 == 0)
    XCoords=np.vstack((r,c)).T
    XCoords = unique_rows(np.sort(XCoords))
    for (x,y) in XCoords:
        for i in range(0,M_Cols-1):
            if M1[x,i]==0 or M1[x,i]==-1: 
                M1[x,i] = M1[y,i]
    return M1
def reduction_Matrix(M2,counter):
    M_rows,M_cols = M2.shape
    dict1={}
    for i,j in enumerate(counter):
        dict1[i]=j
    array2 = np.zeros((len(dict1.keys()),M_cols))
    for i in range(0,len(dict1.keys())):
        if dict1[i]==0:
            continue
        else:
            array2[i]=M2[list(dict1.keys())[i]]
    return array2
def ReducedReadingD(counter,ReadingD):
    ReadingD2 = {}
    a = np.array([
        x
    for x,y in enumerate(counter)
        if y != 0
       
    ])
    for i in a:
        ReadingD2[i]= ReadingD[i]
    return ReadingD2
def ReindexAfterReduction(ReadingD2):
    Reading_Pd = {}
    for x,y in enumerate(ReadingD2.keys()):
        Reading_Pd[x]=ReadingD2[y]
    return Reading_Pd
def Export(Mdis):
    from numpy import savetxt
    savetxt('Dist_Matrix_v2.0_reduction_1e2.csv',Mdis,fmt='%i',delimiter=',')
    #savetxt('Counter1e2.csv',counter,fmt='%i',delimiter=',')
    #savetxt('Merge_list.csv',Merge_list,fmt='%i',delimiter=',')#   continue
def NumberMatrixToString (reduced_array):
        rows,cols = reducedarray.shape
       
        array_rev = np.full((rows,cols),'a')
        for i in range(0,rows):
            for j in range(0,cols):
                if reducedarray[i,j]==1.0:
                    array_rev[i,j]='A'
                elif reducedarray[i,j]==2.0:
                    array_rev[i,j]='C'
                elif reducedarray[i,j]==3.0:
                    array_rev[i,j]='T'
                elif reducedarray[i,j]==4.0:
                    array_rev[i,j]='G'
                elif reducedarray[i,j]==0.0:
                    array_rev[i,j]='n'
                else:
                    array_rev[i,j]= '-'
        return array_rev
def StringMatrixToSeq(array_rev):
    ### input:string matrix in ['a','c','t','g'], output is ['actg'] array
    rows,cols = array_rev.shape
    separator = ''
    seq_array = np.array([
    separator.join(array_rev[i,])
        for i in range(0,rows)
            ])
    return seq_array
def TerminalGapAndNCorrection(array_rev,Counter_Pd):
    seq_rows, seq_cols = array_rev.shape
    array_rev_GapMarked,seq_rows,seq_cols,MissInfoRowCoords = RemoveTerminalGaps(array_rev,seq_rows, seq_cols)
    most_commontype = Counter_Pd['C_Num'].idxmax()
    for i in range(0,len(array_rev_GapMarked)):
        for j in range(0,seq_cols):
            if array_rev_GapMarked[i][j] == '+' or array_rev_GapMarked[i][j] == 'n':
                array_rev_GapMarked[i][j] = array_rev_GapMarked[12][j]
    array_rev =   array_rev_GapMarked
    return array_rev
def RemoveGap(array_rev):
    a,b = array_rev.shape
    array_gap = [
        i 
        for i in range(0,b-1)
            if sum(array_rev[:,i] == '-')== a
    ]
    array_rev = np.delete(array_rev,array_gap,1)
    return array_rev
def FastaRecords(seq_array):
    rows, = seq_array.shape
    separator = ''
    a = np.array([seq_array[i].item() for i in range(0,rows)])
    b = [Seq(i) for i in a]
    records = [
           SeqRecord(Seq(seq), id = str(index), description = "") 
           for index,seq in enumerate(b) 
    ]
    SeqIO.write(records, 'Merged_1e2' ,"fasta")
    return 
def GenerateCounterDataFrame(counter):
    ### Input: Counter that of copyNumbers
    ### NonZeroArray: Counter without 0 copyNumbers
    ### Counter_Pd: DataFrame with Columns: CopyNumber:C_Num and SeqIDs    
    Counnter_Dict = {}
    NonZeroDict = {}
    
    for i,j in enumerate(counter):
        Counnter_Dict[i] = j
    
    for i in range(0,len(Counnter_Dict.values())):
        if counter [i]!=0:
            NonZeroDict[i] = Counnter_Dict[i]
    
    NonZeroArray = np.array(list(NonZeroDict.values()))
    
    Counter_Pd = pd.DataFrame(NonZeroArray)
    Counter_Pd = Counter_Pd.rename(columns = { 0:'C_Num'})
    Counter_Pd['SeqID'] = range(0,len(Counter_Pd))
    Counter_Pd.to_csv('Counter_Pd.csv',sep=',')
    return Counter_Pd
def DeleteIsolateSingleReadings(Mdis,Counter_Pd,T):
    str = f"MdisM{T}" 
    exec("%s = Mdis<T " % (str))
    DistList, = np.where(eval(str).sum(axis=0)==1)
    DistList_CNumblowT = np.array([
        i
    for i in DistList
        if int(Counter_Pd.loc[Counter_Pd["SeqID"]==i,"C_Num"]) == 1
]) 
    Counter_Pd_removeDist = Counter_Pd.drop(DistList_CNumblowT)
  
    return Counter_Pd_removeDist,DistList_CNumblowT
def CoordsOfOnedistInDistMatrix(Mdis):
    
    def unique_rows(a):
        a = np.ascontiguousarray(a)
        unique_a = np.unique(a.view([('', a.dtype)]*a.shape[1]))
        return unique_a.view(a.dtype).reshape((unique_a.shape[0], a.shape[1]))
    
    r,c = np.where(Mdis == 1)
    XCoords=np.vstack((r,c)).T
    XCoords = unique_rows(np.sort(XCoords))
    OneDist_pd = pd.DataFrame(XCoords).rename(columns = { 0:'Seq_ID_i',1:'Seq_ID_j'})
    return OneDist_pd
def FindUniqueOneDistReading(OneDist_pd,Counter_Pd,ReadingD2):
    droplist2 = {}
    for i in range(0,len(Counter_Pd)):
        for j in OneDist_pd.loc[OneDist_pd["Seq_ID_i"]==i,"Seq_ID_j"]:
            if len(OneDist_pd.loc[OneDist_pd["Seq_ID_j"]==j,"Seq_ID_i"]) == 1:
                for ele in ReadingD2[j]:
                    if ele not in ReadingD2[i]:
                        ReadingD2[i].append(ele)
                droplist2[j]= j
                Counter_Pd.loc[Counter_Pd['SeqID'] == i,['C_Num']] += int(Counter_Pd.loc[Counter_Pd['SeqID']==j,'C_Num'])
                droplist2_array = np.array(list(droplist2.keys()))
    try:
        return droplist2_array,Counter_Pd,ReadingD2
    except:
        return [],Counter_Pd,ReadingD2
def Dropthereadings(Counter_Pd,droplistT1_array):
    Counter_Pd = Counter_Pd.drop(droplistT1_array,axis=0)
    for i in droplistT1_array:
        Reading_Pd.pop(i)
    return Counter_Pd,Reading_Pd       
def OneCopyReadingList(Counter_Pd):
    OneCopyDf = Counter_Pd[Counter_Pd["C_Num"]== 1]
    OneCopyDfSeqID = np.array(list(OneCopyDf["SeqID"]))
    return OneCopyDf,OneCopyDfSeqID
def DefineUnCertain(Counter_Pd,T1,T2):
    #Input: T1: lower bond of UnCertainreadings, T2: upperBond of UnCertainReadings
    UnCertainlist = (Counter_Pd[Counter_Pd["C_Num"] >T1]).loc[(
        Counter_Pd[Counter_Pd["C_Num"] >T1])['C_Num']<T2]
    UnCertainID = np.array(list(UnCertainlist['SeqID']))
    return UnCertainlist,UnCertainID

def DefineCertain(Counter_Pd,T1):
    #Input: T1 define the lower limit for certain C_Num
    Certainlist = Counter_Pd[Counter_Pd["C_Num"] >=T1] 
    CertainID = np.array(list(Certainlist['SeqID']))
    return Certainlist,CertainID
def Split_Merge_1copy_to_UnCertain(UnCertainlist,UnCertainID,OneCopyDfSeqID,Mdis,Counter_Pd,Reading_Pd,T):
    Mdis_Pd = pd.DataFrame(Mdis)
    droplist_PartialUnCertain = {}
    
    for i in UnCertainID:
        for j in list((Mdis_Pd.loc[Mdis_Pd[i]==T]).index):
            if j in OneCopyDfSeqID:
                droplist_PartialUnCertain[j] = j
                droplist_PartialUnCertain_array = np.array(list(droplist_PartialUnCertain.keys()))
                UnCertainlist.loc[UnCertainlist['SeqID'] == i,'C_Num'] +=  float(UnCertainlist.loc[UnCertainlist['SeqID'] == i,'C_Num']) /sum([
                float(Counter_Pd.loc[Counter_Pd['SeqID']==k,'C_Num'])
                for k in list(Mdis_Pd.loc[Mdis_Pd[j]==T].index)
                ])
                for ele in Reading_Pd[j]:
                    if ele not in Reading_Pd[i]:
                           Reading_Pd[i].append(ele)   
    try:    
        return UnCertainlist,droplist_PartialUnCertain_array,Reading_Pd
    except:
        return UnCertainlist,[],Reading_Pd
def Split_Merge_1copy_to_certain(Certainlist,CertainID,OneCopyDfSeqID,Mdis,Counter_Pd,Reading_Pd,T):
    droplist_PartialCertain = {}
    Mdis_Pd = pd.DataFrame(Mdis) 
    for i in CertainID:
        for j in list((Mdis_Pd.loc[Mdis_Pd[i]==T]).index):
            if j in OneCopyDfSeqID:
                droplist_PartialCertain[j] = j
                droplist_PartialCertain_array = np.array(list(droplist_PartialCertain.keys()))
                Certainlist.loc[Certainlist['SeqID'] == i,'C_Num'] += float(Certainlist.loc[Certainlist['SeqID'] == i,'C_Num']) /sum([
                float(Counter_Pd.loc[Counter_Pd['SeqID']==k,'C_Num'])
                for k in list(Mdis_Pd.loc[Mdis_Pd[j]==T].index)
                ])
                for ele in Reading_Pd[j]:
                    if ele not in Reading_Pd[i]:
                        Reading_Pd[i].append(ele)                
    try: 
        return Certainlist,droplist_PartialCertain_array,Reading_Pd
    except:
        return Certainlist,[],Reading_Pd


def OneCopyreadingNotJoinedMerging(OneCopyDf,OneCopyDfSeqID,droplist_PartialUnCertain_array, droplist_PartialCertain_array):
    a = np.concatenate((droplist_PartialUnCertain_array, droplist_PartialCertain_array), axis=None)
    ReadingsInvolved = np.unique(a)
    OneCopyNotInvolved_array = OneCopyDfSeqID[~np.isin(OneCopyDfSeqID, ReadingsInvolved)]
    OneCopyNotInvolved_df = OneCopyDf.drop(ReadingsInvolved)
    return OneCopyNotInvolved_df,OneCopyNotInvolved_array
def CoordsOfTdistInDistMatrix(Mdis,T,arrayofInterest):
    
    def unique_rows(a):
        a = np.ascontiguousarray(a)
        unique_a = np.unique(a.view([('', a.dtype)]*a.shape[1]))
        return unique_a.view(a.dtype).reshape((unique_a.shape[0], a.shape[1]))
    
    r,c = np.where(Mdis == T)
    r1 = np.array([
        i
        for i in r
            if i in arrayofInterest 
    ])
    c1 = np.array([
        i
        for i in c
            if i in arrayofInterest 
    ])
    XCoords=np.vstack((r1,c1)).T
    XCoords = unique_rows(np.sort(XCoords))
    TDist_pd = pd.DataFrame(XCoords).rename(columns = { 0:'Seq_ID_i',1:'Seq_ID_j'})
    return TDist_pd
def autodistmerge(Mdis,arrayofInterest,OneCopyNotInvolved_df,OneCopyNotInvolved_array,Reading_Pd,ids,Counter_Pd):
    
    T = 1
    CheckPoint = 1
    while CheckPoint > 0.1:
        T +=1
        TDist_pd = CoordsOfTdistInDistMatrix(Mdis,T,arrayofInterest)
        droplist_Unique_array,Counter_Pd,Reading_Pd = FindUniqueTDistReading(TDist_pd,Counter_Pd,OneCopyNotInvolved_array,Reading_Pd)
        UnCertainlist,UnCertainID = DefineUnCertain(Counter_Pd,5,10)
        Certainlist,CertainID = DefineCertain(Counter_Pd,10)
        UnCertainlist,droplist_PartialUnCertain_array,Reading_Pd = Split_Merge_1copy_to_UnCertain(UnCertainlist,UnCertainID,OneCopyNotInvolved_array,Mdis,Counter_Pd,Reading_Pd,T)
        Certainlist,droplist_PartialCertain_array,Reading_Pd= Split_Merge_1copy_to_certain(Certainlist,CertainID,OneCopyNotInvolved_array,Mdis,Counter_Pd,Reading_Pd,T)
        OneCopyNotInvolved_df,OneCopyNotInvolved_array =  OneCopyreadingNotJoinedMerging(OneCopyNotInvolved_df,OneCopyNotInvolved_array,droplist_PartialUnCertain_array, droplist_PartialCertain_array)
        arrayofInterest = np.concatenate([CertainID, UnCertainID, OneCopyNotInvolved_array])
        CheckPoint = len(OneCopyNotInvolved_array)/len(ids) 
    else:
        print(len(OneCopyNotInvolved_array),'of One Copy Readinds has not involved in merging,','Merging happened up to',T,'dist')
def FindUniqueTDistReading(T_pd,Counter_Pd,OneCopyNotInvolved_array,Reading_Pd):
    droplist_TUnique = {}
    for i in range(0,len(Counter_Pd)):
        for j in T_pd.loc[T_pd["Seq_ID_i"]==i,"Seq_ID_j"]:
            if len(T_pd.loc[T_pd["Seq_ID_i"]==j,"Seq_ID_j"]) == 1:
                if j in OneCopyNotInvolved_array:
                    droplist_TUnique[j]= j
                    Counter_Pd.loc[Counter_Pd['SeqID'] == i,['C_Num']] += float(Counter_Pd.loc[Counter_Pd['SeqID']==j,'C_Num'])
                    droplist_TUnique_array = np.array(list(droplist_TUnique.keys()))
                    for ele in Reading_Pd[j]:
                        if ele not in Reading_Pd[i]:
                            Reading_Pd[i].append(ele)                
    try:
        return droplist_TUnique_array,Counter_Pd,Reading_Pd
    except:
        return [],Counter_Pd,Reading_Pd
def CalChrCount(ids,Df):    

    UniChrPos = np.array( ['chr13','chr14','chr15','chr21','chr22'])
    IDarray = np.array(Df['SeqID'])
    data = np.full((len(IDarray),len(UniChrPos)),0)
    UniChrPos_df = pd.DataFrame(data,columns=UniChrPos)
    UniChrPos_df['SeqID']=IDarray
    for key in Reading_Pd.keys():
        for x,y in enumerate(Reading_Pd[key]):
            try:
                UniChrPos_df.loc[UniChrPos_df['SeqID']==key,[Reading_Pd[key][x][0:5]]] +=1
            except:
                continue
    IDlist = list(UniChrPos_df['SeqID'])
    UniChrPos_df = UniChrPos_df.set_index('SeqID',drop=True)   
    return IDlist,UniChrPos_df
def ExpChrCount(CU_count,C_count):
    columns_CU = list(C_count.columns)
    columns_C =  list(C_count.columns)
    data_CU = np.full((1,len(columns_CU)),0)
    data_C = np.full((1,len(columns_C)),0)
    EmptyRef_CU = pd.DataFrame(data_CU,columns=columns_CU,index=[111111])
    EmptyRef_C = pd.DataFrame(data_C,columns=columns_C,index=[111111])
    CertainAndUnEx = CU_count.append(EmptyRef_CU)
    CertainEx = C_count.append(EmptyRef_C)
    CertainAndUnEx.to_csv(F"LociPosof{len(CertainAndUnEx)}Readings.csv",sep = ',')
    CertainEx.to_csv(F"LociPosof{len(CertainEx)}Readings.csv",sep = ',')
def FastaRecords_Final_CandUnC(MergedSeqRecords_array,IDlist,S):
    rows, = MergedSeqRecords_array.shape
    input_file = '/data/judong/Bash_RibosomalRnaGeneAlignment/AlignmentResults/Ref/ref.fas'
    records = SeqIO.to_dict(SeqIO.parse(input_file, "fasta"))
    ids = list(SeqIO.to_dict(SeqIO.parse(input_file, "fasta")).keys())
    array =[
            y
            for i in range(0,len(ids))
                for y in records[ids[i]].seq
        ]
    rows, = MergedSeqRecords_array.shape
    separator = ''
    a = np.array([MergedSeqRecords_array[i].item() for i in range(0,rows)])
    b = [Seq(i) for i in a]
    c = np.array((IDlist,b),dtype=object).T
    d = np.array([['ref',(records[ids[0]].seq)]],dtype=object)
    e = np.array([['ref',(records[ids[1]].seq)]],dtype=object)
    if S == 18:
        f = np.append(c,d,axis = 0)
    if S == 28:
        f = np.append(c,e,axis = 0)

    
    records2 = [
           SeqRecord(Seq(seq), id = str(index), description = "") 
           for index,seq in f 
    ]
    
    SeqIO.write(records2, 'Final_Merged_CertainandUnCertain' ,"fasta")
    return 
def CertainC_Numlist(Certain_Df,CertainIDlist):
    CertainC_Numlist = np.array([
        Certain_Df['C_Num'][i]
             for i in CertainIDlist   
    ])
    CertainID_CNum = np.array([
    f"ID{CertainIDlist[i]}_C{int(CertainC_Numlist[i])}"
        for i in range(0,len(CertainIDlist))
    ])
    return CertainID_CNum
def FastaRecords_Final_Certain(Certain,CertainID_CNum,S):
    input_file = '/data/judong/Bash_RibosomalRnaGeneAlignment/AlignmentResults/Ref/ref.fas'
    records = SeqIO.to_dict(SeqIO.parse(input_file, "fasta"))
    ids = list(SeqIO.to_dict(SeqIO.parse(input_file, "fasta")).keys())
    array =[
            y
            for i in range(0,len(ids))
                for y in records[ids[i]].seq
        ]
    rows, = Certain.shape
    separator = ''
    a = np.array([Certain[i].item() for i in range(0,rows)])
    b = [Seq(i) for i in a]
    c = np.array((CertainID_CNum,b),dtype=object).T
    d = np.array([['ref',(records[ids[0]].seq)]],dtype=object)
    e = np.array([['ref',(records[ids[1]].seq)]],dtype=object)
    if S == 18:
        d = np.append(c,d,axis = 0)
    if S == 28:
        d = np.append(c,e,axis = 0)

    
    records2 = [
           SeqRecord(Seq(seq), id = str(index), description = "") 
           for index,seq in d 
    ]
    
    SeqIO.write(records2, 'Final_Merged_Certain' ,"fasta")
    return 

def ExportMergeFinal(certaindf,ThreeDistMergingDf):
    certaindf.to_csv('CertainReadings.csv',sep=',')
    a = ThreeDistMergingDf.loc[ThreeDistMergingDf['C_Num']>=5,]
    a.to_csv('CertainandUnCertainReadings.csv',sep=',')

In [3]:
    M,M_rows,M_cols,ids = CreateMatrix()
    ReadingNameArray = CreateReadingNameArray(ids)
    M,M_rows,M_cols,MissInfoRowCoords = RemoveTerminalGaps(M,M_rows, M_cols)
    MissInfoRowCoords = np.sort(MissInfoRowCoords)
    M_WithOutTerminalGapReadings = np.delete(M,(MissInfoRowCoords),axis=0)
    ReMergeM = np.concatenate((M_WithOutTerminalGapReadings,M[MissInfoRowCoords]),axis=0)
    ReorderReadNames = np.delete(ReadingNameArray,(MissInfoRowCoords),axis=0)
    ReorderRN = np.concatenate((ReorderReadNames,ReadingNameArray[MissInfoRowCoords]),axis=0)   
    ReadingD = CreateReadingNameDicts(ReorderRN)
    M_rows,M_cols=ReMergeM.shape
    ErrorMasking(ReMergeM,M_rows,M_cols,0.01)
    M_Num = StringMatrixToNumber (ReMergeM)
    counter,Mdis0,ReadingD = CalculatePairWiseDistance(M_Num,ReadingD)
    M_Num = n_correction(Mdis0,M_Num)
    array2 = reduction_Matrix(M_Num,counter)
    reducedarray = array2[~np.all(array2 == 0, axis=1)]
    array_rev = NumberMatrixToString (reducedarray) 
    ReadingD2 = ReducedReadingD(counter,ReadingD)
    Reading_Pd= ReindexAfterReduction(ReadingD2)
    Mdis = pairwise_distances(reducedarray,metric = CalculatePairWiseDistance_sk)
    Export(Mdis)
    array_rev = NumberMatrixToString (reducedarray)
    array_rev = RemoveGap(array_rev)
    Counter_Pd = GenerateCounterDataFrame(counter)
    array_rev =TerminalGapAndNCorrection(array_rev,Counter_Pd)
    seq_array = StringMatrixToSeq(array_rev)
    FastaRecords(seq_array)
    OneDist_pd =  CoordsOfOnedistInDistMatrix(Mdis)
    droplistT1_array,Counter_Pd,Reading_Pd =FindUniqueOneDistReading(OneDist_pd,Counter_Pd,Reading_Pd) 
    OneCopyDf,OneCopyDfSeqID = OneCopyReadingList(Counter_Pd)
    UnCertainlist,UnCertainID = DefineUnCertain(Counter_Pd,5,10)
    Certainlist,CertainID = DefineCertain(Counter_Pd,10)
    UnCertainlist,droplist_PartialUnCertain_array,Reading_Pd = Split_Merge_1copy_to_UnCertain(UnCertainlist,UnCertainID,OneCopyDfSeqID,Mdis,Counter_Pd,Reading_Pd,1)
    Certainlist,droplist_PartialCertain_array,Reading_Pd= Split_Merge_1copy_to_certain(Certainlist,CertainID,OneCopyDfSeqID,Mdis,Counter_Pd,Reading_Pd,1)
    OneCopyNotInvolved_df,OneCopyNotInvolved_array =  OneCopyreadingNotJoinedMerging(OneCopyDf,OneCopyDfSeqID,droplist_PartialUnCertain_array, droplist_PartialCertain_array)
    arrayofInterest = np.concatenate([CertainID, UnCertainID, OneCopyNotInvolved_array])
    autodistmerge(Mdis,arrayofInterest,OneCopyNotInvolved_df,OneCopyNotInvolved_array,Reading_Pd,ids,Counter_Pd)
    Certain_UnCertain_Df  = Counter_Pd.loc[Counter_Pd['C_Num']>=5]
    Certain_Df = Counter_Pd.loc[Counter_Pd['C_Num']>=10]
    Certain_UnCertain = Certain_UnCertain_Df.index
    Certain = Certain_Df.index
    seq_df = pd.DataFrame(seq_array)
    MergedSeqRecords = seq_df.iloc[Certain_UnCertain]
    Certain_UnCertain = np.array(list(MergedSeqRecords[0]))
    MergedSeqRecords2 = seq_df.iloc[Certain]
    Certain = np.array(list(MergedSeqRecords2[0]))
    IDlist,CU_count =CalChrCount(ids,Certain_UnCertain_Df)
    CertainIDlist,C_count = CalChrCount(ids,Certain_Df)
    CertainID_CNum  = CertainC_Numlist(Certain_Df,CertainIDlist)
    C_count['SeqID'] = CertainID_CNum
    C_count= C_count.set_index('SeqID',drop=True)
    ExpChrCount(CU_count,C_count)
    FastaRecords_Final_CandUnC(Certain_UnCertain,IDlist,18)
    FastaRecords_Final_Certain(Certain,CertainID_CNum,18)
    ExportMergeFinal(Certain_Df,Counter_Pd)   

119  rows reads with terminal gaps have been Masked.
First three position of number Matrix: 3 1 2 Datatype: int64
73  rows reads with terminal gaps have been Masked.
314 of One Copy Readinds has not involved in merging, Merging happened up to 4 dist


In [4]:
len(ids)

3457